# Import

In [ ]:
# Importing pm4py and loading the event log

import pm4py
import pandas as pd

print("Loading event log...")
incident_df = pm4py.read.read_xes("./data/BPI_Challenge_2013_incidents.xes")

filtered_log = pm4py.filter_variants_top_k(incident_df, 5)

# Process Discovery

In [ ]:
# Petri net discovery and visualization
incident_petri_net, initial_markings, final_markings = pm4py.discovery.discover_petri_net_inductive(filtered_log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
pm4py.vis.view_petri_net(incident_petri_net, initial_markings, final_markings, format='png')

In [ ]:
# Process tree discovery and visualization
incident_process_tree = pm4py.discover_process_tree_inductive(filtered_log)
pm4py.view_process_tree(incident_process_tree, format='png')

In [ ]:
# Directly-follows graph discovery and visualization
incident_dfg, start_act, end_act = pm4py.discover_dfg(filtered_log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
pm4py.view_dfg(incident_dfg, start_act, end_act, format='png')

In [ ]:
# BPMN dicovery and visualization
incident_bpmn = pm4py.discover_bpmn_inductive(filtered_log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
pm4py.view_bpmn(incident_bpmn, format='png')


# Conformance checking

In [ ]:
# Token-based replay
replayed_traces = pm4py.conformance_diagnostics_token_based_replay(filtered_log, incident_petri_net, initial_markings, final_markings)
replayed_traces

In [ ]:
# Alignment-based replay
aligned_traces = pm4py.conformance_diagnostics_alignments(filtered_log, incident_petri_net, initial_markings, final_markings)
aligned_traces

In [ ]:
# Footprints
from pm4py.visualization.footprints import visualizer as fp_visualizer
from pm4py.algo.discovery.footprints import algorithm as fp_discovery

footprints_net = fp_discovery.apply(incident_petri_net, initial_markings, final_markings)
gviz = fp_visualizer.apply(footprints_net, parameters={fp_visualizer.Variants.SINGLE.value.Parameters.FORMAT: "png"})
fp_visualizer.view(gviz)

In [ ]:
# Log skeleton
from pm4py.algo.discovery.log_skeleton import algorithm as lsk_discovery
from pm4py.algo.conformance.log_skeleton import algorithm as lsk_conformance

skeleton = lsk_discovery.apply(filtered_log, parameters={lsk_discovery.Variants.CLASSIC.value.Parameters.NOISE_THRESHOLD: 0.0})
conf_result = lsk_conformance.apply(filtered_log, skeleton)
conf_result

In [ ]:
# DECLARE model
declare_model = pm4py.discover_declare(filtered_log)
conf_result = pm4py.conformance_declare(filtered_log, declare_model)
conf_result

In [ ]:
# Temporal profile
from pm4py.algo.discovery.temporal_profile import algorithm as temporal_profile_discovery
from pm4py.algo.conformance.temporal_profile import algorithm as temporal_profile_conformance

temporal_profile = temporal_profile_discovery.apply(filtered_log)
temporal_results = temporal_profile_conformance.apply(filtered_log, temporal_profile)
temporal_results

# Statistics

In [ ]:
# Troughput time
all_cases_duration = pm4py.get_all_case_durations(filtered_log)
all_cases_duration

In [ ]:
# Case arrival and case dispersion ratio
from pm4py.statistics.traces.generic.log import case_arrival

case_arrival_ratio = pm4py.get_case_arrival_average(filtered_log)
case_dispersion_ratio = case_arrival.get_case_dispersion_avg(filtered_log)

print("Arrival ratio:", case_arrival_ratio)
print("Disperion ratio:", case_dispersion_ratio)

In [ ]:
# Eventually-follows graph
efg_graph = pm4py.discover_eventually_follows_graph(filtered_log)
efg_graph

In [ ]:
# Case duration
from pm4py.util import constants
from pm4py.statistics.traces.generic.log import case_statistics
from pm4py.visualization.graphs import visualizer as graphs_visualizer

x, y = case_statistics.get_kde_caseduration(filtered_log, parameters={constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: "time:timestamp"})

gviz = graphs_visualizer.apply_plot(x, y, variant=graphs_visualizer.Variants.CASES)
graphs_visualizer.view(gviz)

In [ ]:
# Event distribution
pm4py.view_events_distribution_graph(filtered_log, distr_type="months", format="png")

# Evaluation

In [ ]:
# Replay fitness
fitness_token_based = pm4py.fitness_token_based_replay(filtered_log, incident_petri_net, initial_markings, final_markings)
fitness_alignment_based = pm4py.fitness_alignments(filtered_log, incident_petri_net, initial_markings, final_markings)

print("Fitness token based")
print(fitness_token_based)

print("Fitness alignment based")
print(fitness_alignment_based)

In [ ]:
# Precision
precision_et_conformance = pm4py.precision_token_based_replay(filtered_log, incident_petri_net, initial_markings, final_markings)
precision_align_et_conformance = pm4py.precision_alignments(filtered_log, incident_petri_net, initial_markings, final_markings)

print("Precision ETConformance:", precision_et_conformance)
print("Precision Alignment-ETConformance", precision_align_et_conformance)

In [ ]:
# Generalization
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator

generalization = generalization_evaluator.apply(filtered_log, incident_petri_net, initial_markings, final_markings)

print("Generalization:", generalization)

In [ ]:
# Simplicity
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator

simplicity = simplicity_evaluator.apply(incident_petri_net)

print("Simplicity:", simplicity)

# Simulation

In [ ]:
# Playout of a Petri net
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator

simulated_log = simulator.apply(incident_petri_net, initial_markings, variant=simulator.Variants.BASIC_PLAYOUT, parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: 50})

# Conversion to data frame has been added for better visualization of the result
simulated_df = pm4py.convert_to_dataframe(simulated_log)
simulated_df

# Social Network Analysis

In [ ]:
# Handover of work
handover_of_work_metric = pm4py.discover_handover_of_work_network(filtered_log)

pm4py.view_sna(handover_of_work_metric)
# view_sna will generate a html file "jupyter_sna_vis.html" which has to be opened in the browser

In [ ]:
# Subcontracting
sub_values = pm4py.discover_subcontracting_network(filtered_log)

pm4py.view_sna(sub_values)
# view_sna will generate a html file "jupyter_sna_vis.html" which has to be opened in the browser

In [ ]:
# Working together
wt_values = pm4py.discover_working_together_network(filtered_log)

pm4py.view_sna(wt_values)
# view_sna will generate a html file "jupyter_sna_vis.html" which has to be opened in the browser

In [ ]:
# Similar activities
similar_values = pm4py.discover_activity_based_resource_similarity(filtered_log)

pm4py.view_sna(similar_values)
# view_sna will generate a html file "jupyter_sna_vis.html" which has to be opened in the browser